In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from tqdm.auto import tqdm

c:\Users\marco\Learning\pybit-trading-bot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("SOLUSDT-1m-220802.csv")
data["timestamp"].apply(lambda ts: datetime.fromisoformat(ts))

0         2022-08-02 00:00:00
1         2022-08-02 00:01:00
2         2022-08-02 00:02:00
3         2022-08-02 00:03:00
4         2022-08-02 00:04:00
                  ...        
1053564   2024-08-02 15:24:00
1053565   2024-08-02 15:25:00
1053566   2024-08-02 15:26:00
1053567   2024-08-02 15:27:00
1053568   2024-08-02 15:28:00
Name: timestamp, Length: 1053569, dtype: datetime64[ns]

In [3]:
X = data[["open", "high", "low", "close", "volume", "turnover"]].values

open = X[:, 0]
high = X[:, 1]
low = X[:, 2]
close = X[:, 3]
volume = X[:, 4]
turnover = X[:, 5]


def sliding_window_view(X: np.ndarray, window: int):
    return np.lib.stride_tricks.sliding_window_view(X, (window, )).squeeze()


def none_if_zero(i):
    return None if i == 0 else i


MAX_WINDOW_SIZE = 43200

X = np.array([close[MAX_WINDOW_SIZE - 1:-MAX_WINDOW_SIZE]])
Y = np.array([open[MAX_WINDOW_SIZE:-MAX_WINDOW_SIZE + 1]])

for window_size in [60, 1440, 10080, 43200]:
    X_open = open[MAX_WINDOW_SIZE - window_size:-MAX_WINDOW_SIZE -
                  window_size + 1]
    X_high = sliding_window_view(
        high[MAX_WINDOW_SIZE - window_size:-MAX_WINDOW_SIZE],
        window_size).max(axis=-1)
    X_low = sliding_window_view(
        low[MAX_WINDOW_SIZE - window_size:-MAX_WINDOW_SIZE],
        window_size).min(axis=-1)
    X_turnover = sliding_window_view(
        turnover[MAX_WINDOW_SIZE - window_size:-MAX_WINDOW_SIZE],
        window_size).sum(axis=-1)
    X_volume = sliding_window_view(
        volume[MAX_WINDOW_SIZE - window_size:-MAX_WINDOW_SIZE],
        window_size).sum(axis=-1)

    Y_close = close[MAX_WINDOW_SIZE + window_size -
                    1:none_if_zero(-MAX_WINDOW_SIZE + window_size)]
    Y_high = sliding_window_view(
        high[MAX_WINDOW_SIZE:none_if_zero(-MAX_WINDOW_SIZE + window_size)],
        window_size).max(axis=-1)
    Y_low = sliding_window_view(
        low[MAX_WINDOW_SIZE:none_if_zero(-MAX_WINDOW_SIZE + window_size)],
        window_size).max(axis=-1)
    Y_volume = sliding_window_view(
        volume[MAX_WINDOW_SIZE:none_if_zero(-MAX_WINDOW_SIZE + window_size)],
        window_size).max(axis=-1)
    Y_turnover = sliding_window_view(
        turnover[MAX_WINDOW_SIZE:none_if_zero(-MAX_WINDOW_SIZE + window_size)],
        window_size).max(axis=-1)

    print(X_open.shape, X_high.shape, X_low.shape, X_turnover.shape,
          X_volume.shape)

    print(Y_close.shape, Y_high.shape, Y_low.shape, Y_turnover.shape,
          Y_volume.shape)

    X = np.concatenate(
        (X, np.array([X_open, X_high, X_low, X_turnover, X_volume])), axis=0)
    Y = np.concatenate(
        (Y, np.array([Y_close, Y_high, Y_low, Y_turnover, Y_volume])), axis=0)

(967170,) (967170,) (967170,) (967170,) (967170,)
(967170,) (967170,) (967170,) (967170,) (967170,)
(967170,) (967170,) (967170,) (967170,) (967170,)
(967170,) (967170,) (967170,) (967170,) (967170,)
(967170,) (967170,) (967170,) (967170,) (967170,)
(967170,) (967170,) (967170,) (967170,) (967170,)
(967170,) (967170,) (967170,) (967170,) (967170,)
(967170,) (967170,) (967170,) (967170,) (967170,)


In [4]:
X = X.T
Y = Y.T

X.shape, Y.shape

((967170, 21), (967170, 21))

In [5]:
import wandb

wandb.init(project="bybit-trading-bot")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rosinusserrano (rosinusserrano-personal). Use `wandb login --relogin` to force relogin


In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\marco\Learning\pybit-trading-bot\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\marco\Learning\pybit-trading-bot\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\marco\Learning\pybit-trading-bot\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
  

In [9]:
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)

X_train = X[:int(0.8 * len(X))]
Y_train = Y[:int(0.8 * len(Y))]
X_test = X[int(0.8 * len(X)):]
Y_test = Y[int(0.8 * len(Y)):]

dataset = TensorDataset(X_train, Y_train)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

model = nn.Sequential(
    nn.Linear(21, 64),
    nn.Dropout(0.8),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.Dropout(0.8),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.Dropout(0.8),
    nn.ReLU(),
    nn.LayerNorm(64),
    nn.Linear(64, 21),
)


def scaled_mean_absolute_error(y_true, y_pred):
    return torch.mean((torch.abs(y_true - y_pred)) / y_true)


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1000):
    with torch.no_grad():
        model.eval()

        testoutput = model(X_test)
        testloss = scaled_mean_absolute_error(Y_test, testoutput)
        trainoutput = model(X_train)
        trainloss = scaled_mean_absolute_error(Y_train, trainoutput)

        model.train()

        print(epoch, trainloss, testloss)

        wandb.log({"testloss": testloss, "trainloss": trainloss})

    for x, y in tqdm(dataloader):
        optimizer.zero_grad()
        output = model(x)
        loss = scaled_mean_absolute_error(y, output)
        loss.backward()
        optimizer.step()

C:\Users\marco\AppData\Local\Temp\ipykernel_15636\1401492337.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
C:\Users\marco\AppData\Local\Temp\ipykernel_15636\1401492337.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(Y, dtype=torch.float32)


0 tensor(1.0022) tensor(1.0004)


100%|██████████| 756/756 [00:08<00:00, 85.94it/s] 


1 tensor(0.6312) tensor(0.8157)


100%|██████████| 756/756 [00:08<00:00, 90.03it/s] 


...

100%|██████████| 756/756 [00:09<00:00, 81.27it/s] 


772 tensor(0.4370) tensor(0.4257)


100%|██████████| 756/756 [00:08<00:00, 84.46it/s] 


773 tensor(0.4374) tensor(0.4263)


 52%|█████▏    | 394/756 [00:03<00:03, 95.71it/s] 

In [ ]:
i = 2
j = 0
for i in range(20):
    print(X_test[i][j])
    print(Y_test[i][j])
    print(model(X_test[i])[j])
    print(Y_test[i] - model(X_test[i]))
    print(scaled_mean_absolute_error(Y_test[i], model(X_test[i])))